## Final Project Submission

Please fill out:
* Student name: Abdulrahman Aber, Benjamin Bai, Charles Pan, Kevin Rivera
* Student pace: Full Time
* Scheduled project review date/time: 
* Instructor name: David Elliott
* Blog post URL:


# Imports

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sqlite3 as sq
conn = sq.connect('./zippedData/im.db/im.db')
cur = conn.cursor()

# Load all source files as dataframes

In [2]:
bom_movie_gross_df = pd.read_csv('./zippedData/bom.movie_gross.csv.gz')

In [3]:
movie_basics_df = pd.read_sql('''select * from movie_basics''', conn)

In [4]:
directors_df = pd.read_sql("""select * from directors""",conn)

In [5]:
known_for_df = pd.read_sql("""select * from known_for""",conn)

In [6]:
movie_akas_df = pd.read_sql("""select * from movie_akas""",conn)

In [7]:
movie_ratings_df = pd.read_sql("""select * from movie_ratings""",conn)

In [8]:
persons_df = pd.read_sql("""select * from persons""",conn)

In [9]:
principals_df = pd.read_sql("""select * from principals""",conn)

In [10]:
writers_df = pd.read_sql("""select * from writers""",conn)

In [11]:
rt_movie_info_df = pd.read_csv('./zippedData/rt.movie_info.tsv.gz', sep='\t')

In [12]:
rt_reviews_df = pd.read_csv('./zippedData/rt.reviews.tsv.gz', sep='\t', encoding= 'unicode_escape')

In [13]:
tmdb_movies_df = pd.read_csv('./zippedData/tmdb.movies.csv.gz')

In [14]:
tn_movie_budgets_df = pd.read_csv('./zippedData/tn.movie_budgets.csv.gz',
                                 parse_dates=['release_date'])

# TN Movie Budgets Clean Up Process

In [15]:
# Remove $ from production_budget, convert to float
tn_movie_budgets_df['production_budget'] = tn_movie_budgets_df['production_budget'].str.replace(',', '').str.replace('$', '').astype(float)

# Remove $ from domestic_gross, convert to float
tn_movie_budgets_df['domestic_gross'] = tn_movie_budgets_df['domestic_gross'].str.replace(',', '').str.replace('$', '').astype(float)

# Remove $ from worldwide_gross, convert to float
tn_movie_budgets_df['worldwide_gross'] = tn_movie_budgets_df['worldwide_gross'].str.replace(',', '').str.replace('$', '').astype(float)

# new column release_year from release_date to filter based on time frame in scope
tn_movie_budgets_df['release_year'] = pd.DatetimeIndex(tn_movie_budgets_df['release_date']).year

# time frame: 2010-Present; SAG definition of theatrical: budget > 2 million
tn_movie_budgets_df = tn_movie_budgets_df[(tn_movie_budgets_df.release_year >= 2010) & (tn_movie_budgets_df.production_budget > 1999999)]

# Merge TN Movie Budgets with IMDB Movie Basics for Genres Analysis

In [16]:
# rename movie_basics column to match tn_movie_budgets
movie_basics_df.rename(columns={"primary_title":"movie"}, inplace = True)

# merge movie_budgets and imdb movie_basics
moviebudgets_moviebasics_df = tn_movie_budgets_df.merge(movie_basics_df, how='inner', on='movie')

# drop columns that are not relevant to Genre analysis
moviebudgets_moviebasics_df.drop(columns = ['id','release_date', 'movie_id', 'original_title', 'runtime_minutes', 'domestic_gross' ],axis=1,inplace=True)

# remove duplicate movies
moviebudgets_moviebasics_df.drop_duplicates(subset='movie', keep='first', inplace=True, ignore_index=False)

# add column for net_return
moviebudgets_moviebasics_df['net_return'] = (moviebudgets_moviebasics_df['worldwide_gross'] - moviebudgets_moviebasics_df['production_budget'])

# add column for ROI as a %
moviebudgets_moviebasics_df['roi_percent'] = ((moviebudgets_moviebasics_df['net_return'] / moviebudgets_moviebasics_df['production_budget'])*100)

# Genres Analysis Visualizations

In [26]:
# budgets_basics_netreturn_df created to get top 100 movies by net_return
budgets_basics_netreturn_df = moviebudgets_moviebasics_df.sort_values(by='net_return',ascending=False).head(100)
# split genres string to list
budgets_basics_netreturn_df['genres'] = budgets_basics_netreturn_df['genres'].str.split(",")
# split genres lists to unique rows for counts of individual genres instead of combinations
budgets_basics_netreturn_df = budgets_basics_netreturn_df.explode('genres')
# means by genre
netreturn_mean_df = budgets_basics_netreturn_df.groupby(['genres']).mean()

,production_budget,worldwide_gross,release_year,start_year,net_return,roi_percent
genres,,,,,,
Action,1.791220e+08,8.955876e+08,2014.576271,2014.576271,7.164656e+08,442.926500
Adventure,1.720154e+08,8.779216e+08,2014.358974,2014.320513,7.059063e+08,469.024810
Animation,1.314783e+08,8.027967e+08,2013.826087,2013.739130,6.713184e+08,598.969647
Biography,8.266667e+07,6.584167e+08,2015.666667,2015.666667,5.757500e+08,888.560449
Comedy,1.191429e+08,7.660813e+08,2014.571429,2014.321429,6.469385e+08,702.441986
Crime,1.730000e+08,9.547264e+08,2013.800000,2013.000000,7.817264e+08,442.812533
Drama,1.090000e+08,7.495741e+08,2014.500000,2013.285714,6.405741e+08,782.378584
Family,1.625556e+08,7.409188e+08,2014.666667,2013.888889,5.783633e+08,407.918792
Fantasy,1.899222e+08,8.489885e+08,2014.333333,2014.166667,6.590663e+08,421.292771


In [28]:
# budgets_basics_roi_df created to get top 100 movies by ROI%
budgets_basics_roi_df = moviebudgets_moviebasics_df.sort_values(by='roi_percent',ascending=False).head(100)
# split genres string to list
budgets_basics_roi_df['genres'] = budgets_basics_roi_df['genres'].str.split(",")
# split genres lists to unique rows for counts of individual genres instead of combinations
budgets_basics_roi_df = budgets_basics_roi_df.explode('genres')
# means by genre
roi_mean_df = budgets_basics_roi_df.groupby(['genres']).mean()
roi_mean_df

,production_budget,worldwide_gross,release_year,start_year,net_return,roi_percent
genres,,,,,,
Action,2.964286e+07,3.326421e+08,2015.428571,2015.285714,3.029992e+08,1124.257164
Adventure,5.362500e+07,6.473327e+08,2015.333333,2015.166667,5.937077e+08,1074.309057
Animation,6.725000e+07,8.432574e+08,2015.333333,2015.166667,7.760074e+08,1197.222125
Biography,2.109091e+07,2.744904e+08,2015.272727,2015.272727,2.533995e+08,1291.034893
Comedy,2.829688e+07,3.502766e+08,2014.093750,2013.843750,3.219798e+08,1297.821099
Crime,1.100000e+07,1.313186e+08,2017.000000,2017.000000,1.203186e+08,1079.692293
Documentary,1.637500e+07,2.228532e+08,2016.750000,2013.500000,2.064782e+08,1244.007954
Drama,1.557600e+07,1.963463e+08,2014.520000,2013.960000,1.807703e+08,1269.266814
Family,1.875000e+07,2.027057e+08,2014.750000,2014.500000,1.839557e+08,1021.465637
